In [1]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [2]:
def pull_from_csv(csv_file, row=3, header=1):
        lwp_ind = str(csv_file).lower().find('lwp')
        lwp = str(csv_file)[lwp_ind:lwp_ind+14]
        df = pd.read_csv(csv_file, header=[header], on_bad_lines='skip')
        try:
            return [lwp,*df.iloc[row].name]
        except:
            return [lwp,*df.iloc[row]]
        
        
        
        
        
def met_ratio(df, up, down, error_method='Cramer', error_label=' Error (SD)', output='both'):
    if not isinstance(up,list):
        up = [up]
    if not isinstance(down,list):
        down = [down]    
   
    up_err_name = [f'{item}{error_label}' for item in up]
    down_err_name = [f'{item}{error_label}' for item in down]
    for item in np.concatenate([up_err_name, down_err_name]).flat:
        if item not in df.columns:
            raise KeyError(f'Cannot find "{item}" in the DataFrame. Please check the given name is correct.')
    
    up_amp = df[up].astype(float).sum(axis=1)
    down_amp = df[down].astype(float).sum(axis=1)
    if error_method.lower() == 'cramer':
        up_err = (df[up_err_name].astype(float)**2).sum(axis=1) ** 0.5
        down_err = (df[down_err_name].astype(float)**2).sum(axis=1) ** 0.5
    elif error_method.lower()=='narrow':
        up_err = df[up_err_name].astype(float).max(axis=1) 
        down_err = df[down_err_name].astype(float).max(axis=1) 
    else:
        raise KeyError('Please choose "Cramer" or "Narrow" for the error_method')
    
    ratio = up_amp / down_amp
    ratio_err = ratio * ((up_err/up_amp)**2 + (down_err/down_amp)**2)**0.5 
    
    if len(up)>1:
        up_label =f'({"+".join(up)})'
    else:
        up_label = f'{"+".join(up)}'
    if len(down)>1:
        down_label = f'({"+".join(down)})'
    else:
        down_label = f'{"+".join(down)}'    
    col_name = f'{up_label}/{down_label}'   
    col_error_name = f'{col_name} Error (SD) {error_method}'
    
    if output=='both':
        return pd.DataFrame({col_name: ratio, col_error_name: ratio_err})
    elif output=='ratio':
        return pd.DataFrame({col_name: ratio})      
    elif output=='error':
        return pd.DataFrame({col_error_name: ratio_err})   
    else:
        raise KeyError('Please choose "ratio", "error" or "both" (default) as output')

In [3]:
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY')
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/Ellie')
inspire = Path('/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA')
missing_inspire = Path('/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY')

data_folder = missing_inspire
basis_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine')

sdat_files = [f for f in sorted(data_folder.rglob('*')) if ("act.sdat" in f.name.lower() and "press_35".lower() in f.name.lower())]
for file in tqdm(sdat_files):
    ref = Path(f'{str(file)[0:-8]}ref.SDAT')
    csv = Path(f'{str(file)[0:-5]}_PRESS_35_WM.csv')
    pdf = Path(f'{str(file)[0:-4]}pdf')
    print(pdf)
    if not csv.is_file():
    # if 1:    
        command = f'tarquin --rows 1 --cols 1 --input {file} --output_pdf {pdf} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}' # with pdf graph
        # command = f'tarquin --rows 1 --cols 1 --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}' # no pdf graph

        print(f'{command}\n') 
        os.system(command)
        

  0%|          | 0/9 [00:00<?, ?it/s]

/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_SV_PRESS_35_WM_13_1_raw_act.pdf
tarquin --rows 1 --cols 1 --input /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_SV_PRESS_35_WM_13_1_raw_act.SDAT --output_pdf /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_SV_PRESS_35_WM_13_1_raw_act.pdf --output_csv /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_SV_PRESS_35_WM_13_1_raw_act_PRESS_35_WM.csv --input_w /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_SV_PRESS_35_WM_13_1_raw_ref.SDAT --basis_csv /Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine


2023-Jan-03 13:02:05 INFO   : TARQUIN 4.3.10 Started
2023-Jan-03 13:02:05 INFO   : Loading data file : /Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MISSING_MRS_ONLY/LWP713_evening/MRS/C1574056_WIP_SV_PRESS_35_WM_13

.............


2023-Jan-03 13:02:05 INFO   : done.
2023-Jan-03 13:02:05 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Glc.csv'

...........................................

...


2023-Jan-03 13:02:31 INFO   : done.
2023-Jan-03 13:02:31 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Gua.csv'
2023-Jan-03 13:02:31 INFO   : done.
2023-Jan-03 13:02:31 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Ins.csv'
2023-Jan-03 13:02:31 INFO   : done.
2023-Jan-03 13:02:31 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Lac.csv'
2023-Jan-03 13:02:31 INFO   : done.
2023-Jan-03 13:02:31 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Lip09.csv'
2023-Jan-03 13:02:31 INFO   : done.
2023-Jan-03 13:02:31 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Lip13a.csv'
2023-Jan-03 13:02:31 INFO   : done.
2023-Jan-03 13:02:31 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Lip13b.csv'
2023-Jan-03 13:02:31 INFO   : done.
2023-Jan-03 13:02:31 INFO   : Simulating '/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_b

............................................................................................................................................................................


2023-Jan-03 13:02:32 INFO   : done.
2023-Jan-03 13:02:32 DEBUG  : Final Beta: 116.86
2023-Jan-03 13:02:32 DEBUG  : Final Phi0: 0.43
2023-Jan-03 13:02:32 DEBUG  : Final Phi1: 0.00
2023-Jan-03 13:02:32 INFO   : FWHM right ppm = 3.016754
2023-Jan-03 13:02:32 INFO   : FWHM left ppm = 3.077881
2023-Jan-03 13:02:32 INFO   : Metabolite FWHM (ppm) = 0.061127
2023-Jan-03 13:02:32 INFO   : Spec noise            = 0.000001
2023-Jan-03 13:02:32 INFO   : sdev noise = 0.000038
2023-Jan-03 13:02:32 INFO   : SNR residual = 4.590813
2023-Jan-03 13:02:32 INFO   : SNR max      = 5.490702
2023-Jan-03 13:02:32 INFO   : Ymax       = 0.000349
2023-Jan-03 13:02:32 INFO   : Ymax metab = 0.000349
2023-Jan-03 13:02:32 INFO   : Fit quality = 1.196019
2023-Jan-03 13:02:32 INFO   : Baseline dev = 0.052262
2023-Jan-03 13:02:32 INFO   : Ymax         = 0.000349
2023-Jan-03 13:02:32 INFO   : BSL         = 0.025487
2023-Jan-03 13:02:32 INFO   : Computing CRLBs.
2023-Jan-03 13:02:32 DEBUG  : Sigma: 9.751950e-13
2023-Jan

100%|██████████| 9/9 [00:27<00:00,  3.11s/it]



Optimisation details
--------------------
l2 norm of error at initial p  = 12.8858
l2 norm of error at final p    = 9.90507
l2 norm of J.'*e at final p    = 115.157
l2 norm of D*p at final p      = 2.27383e-10
number of iterations           = 18
number of function evaluations = 2
number of Jacobian evaluations = 173
stopped by small D*p


TARQUIN has finished. Please use "--view_fit" to inspect the results.


2023-Jan-03 13:02:32 INFO   : TARQUIN Finished

Writing txt results file.
Done.
Writing csv results file.
Done.


In [4]:
csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("press_35" in f.name.lower() and ".csv" in f.name.lower())]



rows_to_process = {
                   'WM': 0,
                   }

for label,row_num in rows_to_process.items():
    data_list = []
    for file in csv_files:
        pulled_data = [*pull_from_csv(file,row=row_num),*pull_from_csv(file,row=row_num+3),*pull_from_csv(file,header=6,row=row_num+1)]
        data_list.append(pulled_data)
        print(f'{pulled_data[0]} has {len(pulled_data)} elements')

    
        print('{')
    print(f'\n{data_list[0] = }')   
    print(f'{len(data_list[0]) = }')
    
    row0 = pull_from_csv(file,row=row_num)
    print(f'\n{row0 = }')   
    print(f'{len(row0) = }')
    
    row3 = pull_from_csv(file,row=row_num+3)
    print(f'\n{row3 = }')   
    print(f'{len(row3) = }')
    
    H6row1 = pull_from_csv(file,header=6,row=row_num+1)
    print(f'\n{H6row1 = }')   
    print(f'{len(H6row1) = }')
    
    
    
    my_columns = list(pd.read_csv(file, header=[1], on_bad_lines='skip'))
    print(f'\n{my_columns=} ')
    print(f'{len(my_columns)=} ')

    my_errors = [item+' Error (SD)' for item in my_columns]
    print(f'\n{my_errors=} ')
    print(f'{len(my_errors)=} ')


    my_diagnostics = pull_from_csv(file,row=0, header=6)    
    print(f'\n{my_diagnostics=} ')
    print(f'{len(my_diagnostics)=} ')

    total_columns = ['LWP', *my_columns, 'Error for LWP', *my_errors,*my_diagnostics ]
    print(f'\nTotal number of columns = {len(total_columns)}  ')
    df = pd.DataFrame(data_list, columns=total_columns)
    
    df.to_excel(f'{label}.xlsx', index=False)
    
    #now calculate ratios
    # df_L2N = met_ratio(df,['Lac', 'Threonine'], 'TNAA')
    # df_L2N_narrow_error = met_ratio(df,['Lac', 'Threonine'], 'TNAA', error_method='Narrow', output='error')
    # df_N2C = met_ratio(df,'TNAA', 'TCho')
    # df_C2C = met_ratio(df,'TCho', 'Cr')
    # df_N2Cr = met_ratio(df,'TNAA', 'Cr')


    # df_ratio = pd.concat((df_L2N, df_N2C, df_C2C, df_N2C, df), axis=1)
    # df_ratio = pd.concat((df_L2N,df_L2N_narrow_error, df_N2C, df_C2C, df_N2Cr, df), axis=1)

    # df.set_index('LWP').to_excel(f'{label}_RATIOS_PRESS_INSPIRE.xlsx')
    df.set_index('LWP').to_excel(f'{label}_PRESS_INSPIRE_03Jan2023.xlsx')



LWP713_evening has 109 elements
{
LWP714_evening has 109 elements
{
LWP715_evening has 109 elements
{
LWP716_evening has 109 elements
{
LWP717_evening has 109 elements
{
LWP718_evening has 109 elements
{
LWP719_evening has 109 elements
{
LWP720_evening has 109 elements
{
LWP721_evening has 109 elements
{

data_list[0] = ['LWP713_evening', '1', '1', '1', '0.000000e+00', '0.000000e+00', '0.000000e+00', '5.885965e+00', '1.872544e+00', '9.393417e-01', '2.277633e+00', '3.480683e+00', '2.799567e+00', '2.012677e+00', '4.998698e+00', '7.651470e-01', '1.049623e+00', '4.753214e+00', '0.000000e+00', '2.610446e+00', '3.987186e+00', '9.838339e-01', '3.612374e+00', '2.353979e+00', '3.353173e+00', '4.944817e+00', '2.283076e+00', '1.060787e+00', '2.432907e-01', '0.000000e+00', '1.659181e+00', '1.261656e+00', '7.227894e+00', '2.000128e+00', '6.280249e+00', '5.036809e+00', '9.349422e+00', '5.963619e+00', 'LWP713_evening', '1', '1', '1', '5.006562e-01', '6.183724e-01', '2.392000e+00', '1.540544e+00', '5.

In [ ]:
dfbb = met_ratio(df,'TNAA', 'TCho', error_method='Narrow', output='error')
dfbb


In [24]:
my_columns

['Row',
 'Col',
 'Slice',
 '-CrCH2',
 'Ala',
 'Asp',
 'Cr',
 'GABA',
 'GPC',
 'Glc',
 'Gln',
 'Glu',
 'Gua',
 'Ins',
 'Lac',
 'Lip09',
 'Lip13a',
 'Lip13b',
 'Lip20',
 'MM09',
 'MM12',
 'MM14',
 'MM17',
 'MM20',
 'NAA',
 'NAAG',
 'PCh',
 'Scyllo',
 'Tau',
 'TNAA',
 'TCho',
 'Glx',
 'TLM09',
 'TLM13',
 'TLM20']

In [25]:
my_columns

['Row',
 'Col',
 'Slice',
 '-CrCH2',
 'Ala',
 'Asp',
 'Cr',
 'GABA',
 'GPC',
 'Glc',
 'Gln',
 'Glu',
 'Gua',
 'Ins',
 'Lac',
 'Lip09',
 'Lip13a',
 'Lip13b',
 'Lip20',
 'MM09',
 'MM12',
 'MM14',
 'MM17',
 'MM20',
 'NAA',
 'NAAG',
 'PCh',
 'Scyllo',
 'Tau',
 'TNAA',
 'TCho',
 'Glx',
 'TLM09',
 'TLM13',
 'TLM20']

In [47]:
33+35+35+2

105

In [48]:
36+36+33

105